In [2]:
print(2 + 2)

4


In [4]:
!pip3 install numpy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 19.8 MB 4.3 MB/s eta 0:00:01     |█████████████████████████████▌  | 18.2 MB 7.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
